In [46]:
import pandas as pd
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import urllib
import pathlib
import datetime

In [52]:
#Lets first connect to the web driver
CHROMEDRIVER_PATH =(r"C:\Users\dell\Downloads\chromedriver_win32 (1)\chromedriver.exe")

In [53]:
# Comman function for init driver
def init_driver(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    pathlib.Path('download').mkdir(parents=True, exist_ok=True) 
    
    preferences = {
        "download.default_directory": os.getcwd() + os.path.sep + '/download',
    }

    chrome_options.add_experimental_option('prefs', preferences)

    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    driver.get(url)
    return driver

In [54]:
def wait_for_page_load(xpath_url):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        pass
    except NoSuchElementException:
        print("Element not found")

In [55]:
product_name = input('What do you want to search ? ')

What do you want to search ? guitar


In [56]:
driver = init_driver('https://www.amazon.in/')

<ipython-input-53-cb6487642f59>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)


In [57]:
# add data to keyword field 
driver.find_element_by_xpath('//input[@id="twotabsearchtextbox"]').send_keys(product_name)

# hit search 
driver.find_element_by_xpath('//input[@id="nav-search-submit-button"]').click()

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:483: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [58]:
page_count = 0 
all_urls = []   
while page_count < 3:
    sleep(5)
    # wait for page load
    wait_for_page_load('//div[@class="a-section a-spacing-medium"]')    

    # finding all boxes 
    all_box = driver.find_elements_by_xpath('//div[@class="a-section a-spacing-medium"]')

    
    # Ittrating over box to find details
    for one_box in all_box:

        try:
            url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
            all_urls.append(url)
        except:
            pass

        
    # Go to next page
    driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
    try:
        driver.find_element_by_xpath('//li[@class="a-last"]').click()
    except:
        driver.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').click()
    page_count += 1

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [60]:
final_data = []
for one_url in all_urls:
    driver.get(one_url)
    
    try:
        sleep(5)
        wait_for_page_load('//span[@id="priceblock_ourprice"]')    


        try:
            brand_name = driver.find_element_by_xpath('//div[@id="productOverview_feature_div"]//span[@class="a-size-base"]').tex
        except:
            brand_name = driver.find_element_by_xpath('//a[@id="bylineInfo"]').text

        product_name = driver.find_element_by_xpath('//h1[@id="title"]').text
        try:
            rating = driver.find_element_by_xpath('//span[@id="acrPopover"]//span//span').get_attribute('innerHTML')
            ratings_count = driver.find_element_by_xpath('//span[@id="acrCustomerReviewText"]').text
        except:
            rating = '-'
            ratings_count = '-'

        try:
            price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]').text
        except:
            try:
                price = driver.find_element_by_xpath('//span[@id="priceblock_dealprice"]').text
            except:
                price = '-'
      
        return_exchange = driver.find_element_by_xpath('//div[@id="RETURNS_POLICY"]').text
        try:
            expected_delivery = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text
        except:
            expected_delivery = '-'
        availability = driver.find_element_by_xpath('//div[@id="availability_feature_div"]').text
        other_details = driver.find_element_by_xpath('//div[@id="feature-bullets"]').text
        product_url = one_url

        final_data.append({
            'brand_name': brand_name,
            'product_name': product_name,
            'rating': rating,
            'ratings_count': ratings_count,
            'price': price,
            'return_exchange': return_exchange,
            'expected_delivery': expected_delivery,
            'availability': availability,
            'other_details': other_details,
            'product_url': product_url,
        })
    except:
        print('Got error in: ',one_url)
        
data = pd.DataFrame(final_data)
data

brand_name  \
0      Visit the Kadence Store   
1       Visit the JUAREZ Store   
2      Visit the Kadence Store   
3       Visit the JUAREZ Store   
4       Visit the JUAREZ Store   
..                         ...   
179             Brand: Generic   
180  Visit the blueberry Store   
181  Visit the blueberry Store   
182              Brand: JUAREZ   
183     Visit the JUAREZ Store   

                                          product_name              rating  \
0    Kadence Guitar Frontier Series, Blue Acoustic ...  4.0 out of 5 stars   
1    Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...  4.0 out of 5 stars   
2    Kadence Frontier Jumbo Semi Acoustic Guitar Wi...  3.9 out of 5 stars   
3    Juârez JRZ38C/WH 6 Strings Acoustic Guitar 38 ...  3.9 out of 5 stars   
4    Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...  4.0 out of 5 stars   
..                                                 ...                 ...   
179  Mini Acoustic Wooden Guitar Red 23 Inch Portab...                   -   
180  Blueberry B-D38, 38 Inch Acoustic Guitar Kit I...  3.7 out of 5 stars   
181  Blueberry B-40, 40" Acoustic Guitar Kit with B...  1.0 out of 5 stars   
182  JUAREZ Arpeggio 39 Inch Acoustic Guitar Kit, S...  3.3 out of 5 stars   
183         JUAREZ Arpeggio Guitar JRA41SP-Q-BL - blue                   -   

      ratings_count      price     return_exchange expected_delivery  \
0       286 ratings  ₹5,999.00  7 Days Replacement     Monday, Oct 4   
1    13,114 ratings  ₹2,649.00  7 Days Replacement   Saturday, Oct 2   
2     1,028 ratings  ₹7,348.00  7 Days Replacement     Monday, Oct 4   
3       481 ratings  ₹2,649.00  7 Days Replacement   Saturday, Oct 2   
4    13,114 ratings  ₹2,649.00  7 Days Replacement   Saturday, Oct 2   
..              ...        ...                 ...               ...   
179               -  ₹1,831.00  7 Days Replacement       Oct 15 - 22   
180      11 ratings  ₹2,795.00  7 Days Replacement   Saturday, Oct 2   
181        1 rating  ₹3,595.00  7 Days Replacement   Saturday, Oct 2   
182       7 ratings  ₹5,044.00  7 Days Replacement   Saturday, Oct 2   
183               -  ₹6,739.00  7 Days Replacement   Saturday, Oct 2   

                            availability  \
0                              In stock.   
1                              In stock.   
2                              In stock.   
3                              In stock.   
4                              In stock.   
..                                   ...   
179  Usually dispatched in 6 to 10 days.   
180                            In stock.   
181                            In stock.   
182                            In stock.   
183                            In stock.   

                                         other_details  \
0    About this item\nProduct Type : Acoustic Guita...   
1    About this item\nBlack Glossy Finish, Number o...   
2    About this item\nQUALITY STRINGS ---The surfac...   
3    About this item\nNumber of Frets: 18, Acoustic...   
4    About this item\nBlack Glossy Finish, Number o...   
..                                                 ...   
179  About this item\n23 inch Mini Wood Acoustic Gu...   
180  About this item\nMaterial: Top: linden wood, S...   
181  About this item\nMaterial: Top: Spruce wood, S...   
182  About this item\nJuarez Arpeggio 39 Inch Acous...   
183  About this item\nJRA41SP-Q-BL\nColor: Blue\nBo...   

                                           product_url  
0    https://www.amazon.in/gp/slredirect/picassoRed...  
1    https://www.amazon.in/gp/slredirect/picassoRed...  
2    https://www.amazon.in/gp/slredirect/picassoRed...  
3    https://www.amazon.in/gp/slredirect/picassoRed...  
4    https://www.amazon.in/Juarez-Acoustic-Cutaway-...  
..                                                 ...  
179  https://www.amazon.in/Acoustic-Wooden-Portable...  
180  https://www.amazon.in/gp/slredirect/picassoRed...  
181  https://www.amazon.in/gp/slredirect/picassoRed...  
182  https://

In [61]:
keywords = ['fruits', 'cars', 'Machine Learning']

In [62]:
def download_image(all_urls):
    for url in all_urls:
        driver.get(url)
        driver.execute_script(f'a = document.createElement("a");a.setAttribute("download","");a.href = "{url}";document.body.appendChild(a);a.click();')

In [63]:
for key in keywords:
    driver.get('https://images.google.com/')
    driver.find_element_by_xpath('//form//input').send_keys(key)
    driver.find_element_by_xpath('//form//input').send_keys(Keys.ENTER)
    
    
    all_src_list = []
    
    while True:
        all_image_box = driver.find_elements_by_xpath('//div[@id="islmp"]//div[@jsname="DeysSe"]/img')
        
        for one_box in all_image_box:
            try:
                one_box.click()
                sleep(1)
                src = driver.find_element_by_xpath('//div[@id="islsp"]//a/img[contains(@src,"https://")]').get_attribute('src')
                all_src_list.append(src)
            except:
                pass

        driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
        if len(all_src_list) > 100:
            break
        
    download_image(all_src_list)

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:483: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1 st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)  # waiting for 3 seconds here
click_btn = driver.find_element_by_xpath("//div[@class='_2QfC02']//button").click()
#giving inputs to search bar
search_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")
search_bar.send_keys("Smartphones")
search_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
search_btn.click()
time.sleep(5)
# fetching urls of every smartphone on Flipkart
urls = []
for i in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
    urls.append(i.get_attribute("href"))

# getting the required data inside the empty lists
brand_names = []
name = []
color = []
RAM = []
storage = []
prim_cam = []
sec_cam = []
Display_size = []
display_resolution = []
processor = []
processor_core = []
battery_cap = []
prices = []
product_url = []

# fetching battery capacity
for i in driver.find_elements_by_xpath("//div[@class='fMghEO']"):
    try:
        bat_cap = driver.find_element_by_xpath("//ul[@class='_1xgFaf']//li[4]")
        battery_cap.append(bat_cap.text)
    except NoSuchElementException:
        battery_cap.append('-')
for i in urls:
    driver.get(i)
    time.sleep(3)
    
    # fetching brand names
    try:
        br_name = driver.find_element_by_xpath("/html/body/div/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a")
        brand_names.append(br_name.text.replace('Mobiles',''))
    except NoSuchElementException:
        brand_names.append('-')
    # fetching the smartphone names
    try:
        smt_name = driver.find_element_by_xpath("//h1[@class='yhB1nd']//span")
        name.append(smt_name.text)
    except NoSuchElementException:
        name.append('-')
    
    # fetching colors of smartphone
    try:
        clr = driver.find_element_by_xpath("//table[@class='_14cfVK']//tr[4]//td[2]")
        color.append(clr.text)
    except NoSuchElementException:
        color.append('-')
    time.sleep(2)
    
    # getting the read more button
    try:
        read_more_btn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    except NoSuchElementException:
        pass
    time.sleep(3)
    
    # fetching the display size
    try:
        disp_size = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[2]//tr[1]//td[2]")
        Display_size.append(disp_size.text)
    except NoSuchElementException:
        Display_size.append
        
    # fetching the display resolution
    try:
        disp_res = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[2]//tr[2]//td[2]")
        display_resolution.append(disp_res.text)
    except NoSuchElementException:
        display_resolution.append('-')
        
    # fetching the processor information
    try:
        pro = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[3]//tr[2]//td[2]")
        processor.append(pro.text)
    except NoSuchElementException:
        processor.append('-')
        
    # fetching the processor and number of core details
    try:
        pro_core = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[3]//tr[3]//td[2]")
        processor_core.append(pro_core.text)
    except NoSuchElementException:
        processor_core.append('-')
        
    # fetching the storage/ROM details
    try:
        rom = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[4]//tr[1]//td[2]")
        storage.append(rom.text)
    except NoSuchElementException:
        storage.append('-')
        
    # fetching the RAM information
    try:
        ram = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[4]//tr[2]//td[2]")
        RAM.append(ram.text)
    except NoSuchElementException:
        RAM.append('-')
    time.sleep(2)
    
    # fetching the primary camera information
    try:
        p_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[5]//tr[2]//td[2]")
        prim_cam.append(p_cam.text)
    except NoSuchElementException:
        prim_cam.append('-')
        
    # fetching the secondary camera information
    try:
        s_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[5]//tr[5]//td[2]")
        sec_cam.append(s_cam.text)
    except NoSuchElementException:
        sec_cam.append('-')        
   
    # fetching the price of the smartphone
    try:
        price = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
        prices.append(price.text.replace('₹','Rs. '))
    except NoSuchElementException:
        prices.append('-')
    
    
# creating a dataframe now with the obtainined details
data = list(zip(brand_names,name,color,RAM,storage,prim_cam,sec_cam,Display_size,display_resolution,processor,
                processor_core,battery_cap,prices,urls))
df = pd.DataFrame(data, columns = ["Brand Name","Smartphone Name","Color","RAM","Storage/ROM","Primary Camera",
                                   "Secondary Camera","Display Size","Display Resolution","Processor","Processor-Core",
                                   "Battery Capacity","Price","Product Url"])
df

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [75]:

driver.get('https://www.google.com/maps/')

In [76]:
city = input('Enter city: ')

Enter city: banka


In [77]:
driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(city)
driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(Keys.CONTROL + Keys.ENTER)

In [78]:
current_url = driver.current_url

In [79]:
latitude, longitude, _ = current_url.split('@')[1].split('/')[0].split(',')

In [80]:
print('latitude: ',latitude)
print('longitude: ',longitude)

latitude:  27.141237
longitude:  80.8833819


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [81]:
driver.get('https://trak.in/india-startup-funding-investment-2015/')

In [82]:
all_page_length = driver.find_elements_by_xpath('//div[@class="dataTables_length"]//select')

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [83]:
for one_page_length in all_page_length:
    select = Select(one_page_length)
    select.select_by_visible_text('100')

In [84]:
all_table = pd.read_html(driver.page_source)

In [85]:
df = pd.concat(all_table)
df

Sr. No. Date (dd/mm/yyyy)     Startup Name   Industry / Vertical  \
0       1.0        01/04/2021           BYJU’S              Edu-tech   
1       2.0        05/04/2021           Meesho            E-commerce   
2       3.0        14/04/2021           Swiggy  Online Food Delivery   
3       4.0        07/04/2021            Groww               FinTech   
4       5.0        14/04/2021          Beldara            E-commerce   
..      ...               ...              ...                   ...   
31     32.0        26/07/2018    TheCapitalNet              Fin-Tech   
32     33.0        30/07/2018           Shuttl     Consumer Internet   
33     34.0        30/07/2018         Cure Fit     Consumer Internet   
34     35.0        31/07/2018  Five Star Group              Fin-Tech   
35     36.0        31/07/2018      Healthsignz            Healthcare   

                            Sub-Vertical City / Location  \
0                        Online tutoring       Bengaluru   
1              Online reselling platform       Bengaluru   
2                   Online Food Delivery       Bengaluru   
3                    Investment platform       Bengaluru   
4                 Global B2B marketplace          Mumbai   
..                                   ...             ...   
31  Unified Global Investments Ecosystem       Hyderabad   
32              Bus Aggregation Platform        Gurugram   
33           Health and fitness Platform       Bengaluru   
34     Non-Bank Lending Platform For SME         Chennai   
35        Health and Well being Platform       Hyderabad   

                                      Investors' Name      Investment Type  \
0                                     Innoven Capital             Series F   
1                              SoftBank Vision Fund 2             Series E   
2   Amansa Holdings, Carmignac, Falcon Edge Capita...             Series J   
3          MC Global Edtech, B Capital, Baron, others             Series D   
4                            Hindustan Media Ventures              Venture   
..                                                ...                  ...   
31              Lindwall Family Investments LLC (LFI)       Private Equity   
32                Amazon Alexa Fund & Dentsu Ventures       Private Equity   
33  IDG Ventures, Accel Partners, Kalaari Capital ...       Private Equity   
34  TPG, Norwest Venture Partners, Sequoia Capital...       Private Equity   
35                                     Kantilal Patel  Seed/ Angel Funding   

   Amount (In USD)  Unnamed: 9  
0        460000000         NaN  
1        300000000         NaN  
2        343000000         NaN  
3         83000000         NaN  
4          7400000         NaN  
..             ...         ...  
31          500000         NaN  
32         1.1e+07         NaN  
33         1.2e+08         NaN  
34           1e+08         NaN  
35           5e+06         NaN  

[616 rows x 10 columns]

In [86]:
df['Date (dd/mm/yyyy)'] = pd.to_datetime(df['Date (dd/mm/yyyy)'],errors='coerce')

In [87]:
df.dtypes

Sr. No.                       float64
Date (dd/mm/yyyy)      datetime64[ns]
Startup Name                   object
Industry / Vertical            object
Sub-Vertical                   object
City / Location                object
Investors' Name                object
Investment Type                object
Amount (In USD)                object
Unnamed: 9                    float64
dtype: object

In [88]:
df

Sr. No. Date (dd/mm/yyyy)     Startup Name   Industry / Vertical  \
0       1.0        2021-01-04           BYJU’S              Edu-tech   
1       2.0        2021-05-04           Meesho            E-commerce   
2       3.0        2021-04-14           Swiggy  Online Food Delivery   
3       4.0        2021-07-04            Groww               FinTech   
4       5.0        2021-04-14          Beldara            E-commerce   
..      ...               ...              ...                   ...   
31     32.0        2018-07-26    TheCapitalNet              Fin-Tech   
32     33.0        2018-07-30           Shuttl     Consumer Internet   
33     34.0        2018-07-30         Cure Fit     Consumer Internet   
34     35.0        2018-07-31  Five Star Group              Fin-Tech   
35     36.0        2018-07-31      Healthsignz            Healthcare   

                            Sub-Vertical City / Location  \
0                        Online tutoring       Bengaluru   
1              Online reselling platform       Bengaluru   
2                   Online Food Delivery       Bengaluru   
3                    Investment platform       Bengaluru   
4                 Global B2B marketplace          Mumbai   
..                                   ...             ...   
31  Unified Global Investments Ecosystem       Hyderabad   
32              Bus Aggregation Platform        Gurugram   
33           Health and fitness Platform       Bengaluru   
34     Non-Bank Lending Platform For SME         Chennai   
35        Health and Well being Platform       Hyderabad   

                                      Investors' Name      Investment Type  \
0                                     Innoven Capital             Series F   
1                              SoftBank Vision Fund 2             Series E   
2   Amansa Holdings, Carmignac, Falcon Edge Capita...             Series J   
3          MC Global Edtech, B Capital, Baron, others             Series D   
4                            Hindustan Media Ventures              Venture   
..                                                ...                  ...   
31              Lindwall Family Investments LLC (LFI)       Private Equity   
32                Amazon Alexa Fund & Dentsu Ventures       Private Equity   
33  IDG Ventures, Accel Partners, Kalaari Capital ...       Private Equity   
34  TPG, Norwest Venture Partners, Sequoia Capital...       Private Equity   
35                                     Kantilal Patel  Seed/ Angel Funding   

   Amount (In USD)  Unnamed: 9  
0        460000000         NaN  
1        300000000         NaN  
2        343000000         NaN  
3         83000000         NaN  
4          7400000         NaN  
..             ...         ...  
31          500000         NaN  
32         1.1e+07         NaN  
33         1.2e+08         NaN  
34           1e+08         NaN  
35           5e+06         NaN  

[616 rows x 10 columns]

In [89]:
start_time = datetime.datetime(2020, 7, 1)
end_time = datetime.datetime(2020, 9, 30)

In [90]:
df = df[(start_time < df['Date (dd/mm/yyyy)']) & (df['Date (dd/mm/yyyy)'] < end_time )].sort_values('Date (dd/mm/yyyy)')
df

Sr. No. Date (dd/mm/yyyy)               Startup Name  \
3       4.0        2020-07-04                     DeHaat   
8       9.0        2020-07-08                FreshToHome   
5       6.0        2020-07-09                    Melorra   
6       7.0        2020-07-09                        1mg   
9      10.0        2020-07-13  Open Appliances Pvt. Ltd.   
4       5.0        2020-07-13                   Mystifly   
8       9.0        2020-07-14                       FLYX   
3       4.0        2020-07-14                    goDutch   
0       1.0        2020-07-15                   Flipkart   
7       8.0        2020-07-15                  PumPumPum   
2       3.0        2020-07-16                       Crio   
1       2.0        2020-07-16                    Vedantu   
5       6.0        2020-08-04                  Fingerlix   
2       3.0        2020-08-04                      Zupee   
0       1.0        2020-08-09                     Byju’s   
10     11.0        2020-08-09            Cell Propulsion   
9      10.0        2020-08-13                    Eduvanz   
2       3.0        2020-08-13                  HungerBox   
1       2.0        2020-08-13                    Medlife   
6       7.0        2020-08-14                       Niyo   
0       1.0        2020-08-15                     Practo   
8       9.0        2020-08-31                       Apna   
7       8.0        2020-08-31                      mfine   
0       1.0        2020-09-01                     BYJU’S   
2       3.0        2020-09-01                  Mamaearth   
0       1.0        2020-09-04                    Vedantu   
5       6.0        2020-09-07               JetSynthesys   
4       5.0        2020-09-09                Hippo Video   
2       3.0        2020-09-09                     Qshala   

                       Industry / Vertical  \
3                                 AgriTech   
8                               E-commerce   
5                               E-commerce   
6                               E-commerce   
9                   Information Technology   
4                      Airfare Marketplace   
8                               OTT Player   
3                                  FinTech   
0                               E-commerce   
7                        Automotive Rental   
2                                  EduTech   
1                                  EduTech   
5                            Food Delivery   
2                                   Gaming   
0                                  EduTech   
10                              Automobile   
9                                  FinTech   
2                                 FoodTech   
1                               E-commerce   
6                                  FinTech   
0                               HealthTech   
8                          Human Resources   
7                               HealthTech   
0                                   E-Tech   
2                               E-commerce   
0                                  EduTech   
5                 Gaming and Entertainment   
4   Video Customer Experience(CX) Platform   
2                                  EduTech   

                                         Sub-Vertical  \
3                                            AgriTech   
8                                       Food Delivery   
5                                Online Jewelry Store   
6                                     Online Pharmacy   
9               Internet-of-Things Security Solutions   
4   Ticketing, Airline Retailing, and Post-Ticketi...   
8                            Streaming Social Network   
3                                      Group Payments   
0                                          E-commerce   
7                           Used Car-leasing platform   
2                    Learning Platform for Developers   
1                                     Online Tutoring   
5                                  Ready-to-Cook Food   
2                       

In [91]:
df.reset_index()

index  Sr. No. Date (dd/mm/yyyy)               Startup Name  \
0       3      4.0        2020-07-04                     DeHaat   
1       8      9.0        2020-07-08                FreshToHome   
2       5      6.0        2020-07-09                    Melorra   
3       6      7.0        2020-07-09                        1mg   
4       9     10.0        2020-07-13  Open Appliances Pvt. Ltd.   
5       4      5.0        2020-07-13                   Mystifly   
6       8      9.0        2020-07-14                       FLYX   
7       3      4.0        2020-07-14                    goDutch   
8       0      1.0        2020-07-15                   Flipkart   
9       7      8.0        2020-07-15                  PumPumPum   
10      2      3.0        2020-07-16                       Crio   
11      1      2.0        2020-07-16                    Vedantu   
12      5      6.0        2020-08-04                  Fingerlix   
13      2      3.0        2020-08-04                      Zupee   
14      0      1.0        2020-08-09                     Byju’s   
15     10     11.0        2020-08-09            Cell Propulsion   
16      9     10.0        2020-08-13                    Eduvanz   
17      2      3.0        2020-08-13                  HungerBox   
18      1      2.0        2020-08-13                    Medlife   
19      6      7.0        2020-08-14                       Niyo   
20      0      1.0        2020-08-15                     Practo   
21      8      9.0        2020-08-31                       Apna   
22      7      8.0        2020-08-31                      mfine   
23      0      1.0        2020-09-01                     BYJU’S   
24      2      3.0        2020-09-01                  Mamaearth   
25      0      1.0        2020-09-04                    Vedantu   
26      5      6.0        2020-09-07               JetSynthesys   
27      4      5.0        2020-09-09                Hippo Video   
28      2      3.0        2020-09-09                     Qshala   

                       Industry / Vertical  \
0                                 AgriTech   
1                               E-commerce   
2                               E-commerce   
3                               E-commerce   
4                   Information Technology   
5                      Airfare Marketplace   
6                               OTT Player   
7                                  FinTech   
8                               E-commerce   
9                        Automotive Rental   
10                                 EduTech   
11                                 EduTech   
12                           Food Delivery   
13                                  Gaming   
14                                 EduTech   
15                              Automobile   
16                                 FinTech   
17                                FoodTech   
18                              E-commerce   
19                                 FinTech   
20                              HealthTech   
21                         Human Resources   
22                              HealthTech   
23                                  E-Tech   
24                              E-commerce   
25                                 EduTech   
26                Gaming and Entertainment   
27  Video Customer Experience(CX) Platform   
28                                 EduTech   

                                         Sub-Vertical  \
0                                            AgriTech   
1                                       Food Delivery   
2                                Online Jewelry Store   
3                                     Online Pharmacy   
4               Internet-of-Things Security Solutions   
5   Ticketing, Airline Retailing, and Post-Ticketi...   
6                            Streaming Social Network   
7                                      Group Payments   
8                                          E-commerce   
9                           Used Car-leasi

# 7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [92]:
driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html')

In [93]:
all_laptop_title = driver.find_elements_by_xpath('//div[@class="TopNumbeHeading sticky-footer"]/a')

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [94]:
all_name = []
for one_title in all_laptop_title:
    all_name.append(one_title.text)
    
name_series = pd.Series(all_name)
name_series

<ipython-input-94-e7336e1e8f16>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  name_series = pd.Series(all_name)


Series([], dtype: float64)

In [95]:
all_specs = driver.find_elements_by_xpath('//div[@class="Spcs-details"]')

In [96]:
all_spec_list = []
for one_spec in all_specs:

    os_name = one_spec.find_element_by_xpath('.//td[text()="OS"]/parent::*/td[3]').text
    display = one_spec.find_element_by_xpath('.//td[text()="Display"]/parent::*/td[3]').text
    processor = one_spec.find_element_by_xpath('.//td[text()="Processor"]/parent::*/td[3]').text
    memory = one_spec.find_element_by_xpath('.//td[text()="Memory"]/parent::*/td[3]').text
    weight = one_spec.find_element_by_xpath('.//td[text()="Weight"]/parent::*/td[3]').text
    dimension = one_spec.find_element_by_xpath('.//td[text()="Dimension"]/parent::*/td[3]').text
    graphics_processor = one_spec.find_element_by_xpath('.//td[text()="Graphics Processor"]/parent::*/td[3]').text
    
    all_spec_list.append({
        'os' : os_name,
        'display' : display,
        'processor' : processor,
        'memory' : memory,
        'weight' : weight,
        'dimension' : dimension,
        'graphics_processor' : graphics_processor,
    })
    
df = pd.DataFrame(all_spec_list)
df

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


os              display  \
0  Windows 10 Home  17.3" (1920 x 1080)   
1  Windows 10 Home  15.6" (3840 x 2160)   
2  Windows 10 Home  15.6" (1920 x 1080)   
3  Windows 10 Home    14" (1920 x 1080)   
4   Windows 10 Pro  15.6" (1920 x 1080)   
5       Windows 10  15.6" (3840 x 1100)   
6  Windows 10 Home  15.6" (1920 x 1080)   

                                           processor                  memory  \
0   10th Generation Intel® Core™ i7-10700 | 2.90 GHz   1 TB SSD/16 GBGB DDR4   
1       10th Generation Intel® Core™ i9-10980HK | NA   1 TB SSD/16 GBGB DDR4   
2                      AMD Ryzen™ 9 5900HX | 3.3 GHz   1 TB SSD/16 GBGB DDR4   
3               AMD 3rd Generation Ryzen 9 | 3.3 GHz   1 TB SSD/16 GBGB DDR4   
4  10th Generation Intel® Core™ i5-10300H | 2.50 GHz   1 TB SSD/16 GBGB DDR4   
5                 Intel Core i7 10th Gen 10875H | NA  512 GB SSD/4 GBGB DDR4   
6                  AMD Ryzen™ 5-5500U hexa-core | NA  512 GB SSD/8 GBGB DDR4   

  weight                dimension             graphics_processor  
0    4.1   27.65 x 402.6 x 319.14        Intel® UHD Graphics 630  
1     NA                       NA                             NA  
2   2.30       35.4 x 25.9 x 2.26      NVIDIA® GeForce RTX™ 3070  
3   1.65        32.5 x 22.1 x 1.8        NVIDIA GeForce RTX 2060  
4    2.3  363.06 x 259.61 x 23.57  NVIDIA® GeForce® GTX 1650 4GB  
5    2.4  268.30 x 360.00 x 20.90  NVIDIA GeForce RTX 2070 Max-Q  
6   2.15       2.29 x 36.3 x 25.4      NVIDIA® GeForce® GTX 1650

In [97]:
df['name'] = name_series
df

os              display  \
0  Windows 10 Home  17.3" (1920 x 1080)   
1  Windows 10 Home  15.6" (3840 x 2160)   
2  Windows 10 Home  15.6" (1920 x 1080)   
3  Windows 10 Home    14" (1920 x 1080)   
4   Windows 10 Pro  15.6" (1920 x 1080)   
5       Windows 10  15.6" (3840 x 1100)   
6  Windows 10 Home  15.6" (1920 x 1080)   

                                           processor                  memory  \
0   10th Generation Intel® Core™ i7-10700 | 2.90 GHz   1 TB SSD/16 GBGB DDR4   
1       10th Generation Intel® Core™ i9-10980HK | NA   1 TB SSD/16 GBGB DDR4   
2                      AMD Ryzen™ 9 5900HX | 3.3 GHz   1 TB SSD/16 GBGB DDR4   
3               AMD 3rd Generation Ryzen 9 | 3.3 GHz   1 TB SSD/16 GBGB DDR4   
4  10th Generation Intel® Core™ i5-10300H | 2.50 GHz   1 TB SSD/16 GBGB DDR4   
5                 Intel Core i7 10th Gen 10875H | NA  512 GB SSD/4 GBGB DDR4   
6                  AMD Ryzen™ 5-5500U hexa-core | NA  512 GB SSD/8 GBGB DDR4   

  weight                dimension             graphics_processor  name  
0    4.1   27.65 x 402.6 x 319.14        Intel® UHD Graphics 630   NaN  
1     NA                       NA                             NA   NaN  
2   2.30       35.4 x 25.9 x 2.26      NVIDIA® GeForce RTX™ 3070   NaN  
3   1.65        32.5 x 22.1 x 1.8        NVIDIA GeForce RTX 2060   NaN  
4    2.3  363.06 x 259.61 x 23.57  NVIDIA® GeForce® GTX 1650 4GB   NaN  
5    2.4  268.30 x 360.00 x 20.90  NVIDIA GeForce RTX 2070 Max-Q   NaN  
6   2.15       2.29 x 36.3 x 25.4      NVIDIA® GeForce® GTX 1650   NaN

# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [98]:
driver.get('https://www.forbes.com/billionaires/')

In [99]:
all_boxes = driver.find_elements_by_xpath('//div[@class="table-body"]/div/div[contains(@class,"table-row ")]')

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [100]:
all_list = []
for one_box in all_boxes:

    rank = one_box.find_element_by_xpath('.//div[@class="rank"]').text
    name = one_box.find_element_by_xpath('.//div[@class="personName"]').text
    net_worth = one_box.find_element_by_xpath('.//div[@class="netWorth"]').text
    age = one_box.find_element_by_xpath('.//div[@class="age"]').text
    citizenship = one_box.find_element_by_xpath('.//div[@class="countryOfCitizenship"]').text
    source = one_box.find_element_by_xpath('.//div[@class="source"]').text
    industry = one_box.find_element_by_xpath('.//div[@class="category"]').text
    
    all_list.append({
        'rank': rank,
        'name': name,
        'net_worth': net_worth,
        'age': age,
        'citizenship': citizenship,
        'source': source,
        'industry': industry,
    })
df = pd.DataFrame(all_list)
df

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


rank                      name net_worth age    citizenship  \
0      1.                Jeff Bezos    $177 B  57  United States   
1      2.                 Elon Musk    $151 B  49  United States   
2      3.  Bernard Arnault & family    $150 B  72         France   
3      4.                Bill Gates    $124 B  65  United States   
4      5.           Mark Zuckerberg     $97 B  36  United States   
..    ...                       ...       ...  ..            ...   
195  195.           Harry Triguboff   $11.2 B  88      Australia   
196  197.     Leonid Fedun & family   $11.1 B  65         Russia   
197  197.                 Eyal Ofer   $11.1 B  70         Israel   
198  197.              Evan Spiegel   $11.1 B  30  United States   
199  200.     Luis Carlos Sarmiento     $11 B  88       Colombia   

                    source               industry  
0                   Amazon             Technology  
1            Tesla, SpaceX             Automotive  
2                     LVMH       Fashion & Retail  
3                Microsoft             Technology  
4                 Facebook             Technology  
..                     ...                    ...  
195            real estate            Real Estate  
196                    oil                 Energy  
197  real estate, shipping            Diversified  
198               Snapchat             Technology  
199                banking  Finance & Investments  

[200 rows x 7 columns]

# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [101]:
driver.get('https://www.youtube.com/watch?v=kTJczUoc26U')

In [102]:
wait_for_page_load('//ytd-comment-thread-renderer')

# Loading all comments
i = 1 
while True:
    i += 500
    driver.execute_script(f"window.scrollTo(0, {i});")
        
    all_box = driver.find_elements_by_xpath('//ytd-comment-thread-renderer')
    
    sleep(1)
    
    if len(all_box) > 500:
        break

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [103]:
all_list = []
for one_box in all_box:
    comment = one_box.find_element_by_xpath('.//ytd-expander[@id="expander"]').text
    upvote = one_box.find_element_by_xpath('.//span[@id="vote-count-middle"]').text
    time = one_box.find_element_by_xpath('.//div[@id="header-author"]//yt-formatted-string/a').text
    
    all_list.append({
        'comment' : comment,
        'upvote' : upvote,
        'time' : time
    })
    
df = pd.DataFrame(all_list)
df

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


comment upvote          time
0                                                  🔥🔥🔥   130K  2 months ago
1    I can't sing this song properly but still hear...    319     1 day ago
2                                Love ❤️ from India 🇮🇳    601     1 day ago
3    I do the same thing I told you that I never wo...    455  18 hours ago
4    Жду времени, чтобы пройти мимо.\nНадеюсь, вете...     14  12 hours ago
..                                                 ...    ...           ...
515  this hits hard tbf, kid laroi is getting bette...    608  2 months ago
516                                       NICE SONG :)      1     1 day ago
517                       You are my favorite musician            1 day ago
518                             Nice i loved this song          3 hours ago
519          Songs in the 2000s are still the best ;-;         14 hours ago

[520 rows x 3 columns]

# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [104]:
driver.get('https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3')

In [106]:
final_list = []
page_urls = []


while True:
    sleep(5)
    wait_for_page_load('//div[@class="property-card" and @data-v-62f28bda=""]')
    all_boxs = driver.find_elements_by_xpath('//div[@class="property-card" and @data-v-62f28bda=""]')
    for one_box in all_boxs:
        hostel_name = one_box.find_element_by_xpath('.//h2/a').text
        distance_from_city = one_box.find_element_by_xpath('.//a/span').text
        try:
            ratings = one_box.find_element_by_xpath('.//div[contains(@class,"score")]').text
        except:
            ratings = 'No rating'
            
        total_reviews = one_box.find_element_by_xpath('.//div[contains(@class,"reviews")]').text
        overall_reviews = one_box.find_element_by_xpath('.//div[contains(@class,"keyword")]').text
        privates_from_price = one_box.find_elements_by_xpath('.//div[contains(@class,"prices-col")]/a/div')[0].text
        dorms_from_price  = one_box.find_elements_by_xpath('.//div[contains(@class,"prices-col")]/a/div')[1].text
        facilities = one_box.find_element_by_xpath('.//div[contains(@class,"facilities")]').text.split('\n')
        
        
        page_url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
        page_urls.append(page_url)

        final_list.append({
            'hostel_name' : hostel_name,
            'distance_from_city' : distance_from_city,
            'ratings' : ratings,
            'total_reviews' : total_reviews,
            'overall_reviews' : overall_reviews,
            'privates_from_price' : privates_from_price,
            'dorms_from_price' : dorms_from_price,
            'facilities' : facilities,

        })
        
        
    if len(driver.find_elements_by_xpath('//div[contains(@class,"pagination-next disabled")]')) == 0:
        # we have more pages go to next
        driver.find_element_by_xpath('//div[contains(@class,"pagination-next")]').click()
    else:
        # we are at last page
        break
            

df = pd.DataFrame(final_list)
df

C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:392: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:425: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\dell\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:483: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instea

hostel_name  \
0               PubLove @ The Rose & Crown   
1             The Finnish Church in London   
2                  Saint James Backpackers   
3      PubLove @ The Green Man, Paddington   
4                        Mornington Camden   
..                                     ...   
80  Best Western Boltons London Kensington   
81                        Park Hotel Essex   
82                         Cranbrook Hotel   
83                              St. Athans   
84                        Aron Guest House   

                             distance_from_city    ratings  \
0               Hostel - 1.6km from city centre        7.9   
1               Hostel - 4.5km from city centre         10   
2               Hostel - 5.5km from city centre        7.4   
3               Hostel - 4.3km from city centre        6.2   
4               Hostel - 4.1km from city centre         10   
..                                          ...        ...   
80               Hotel - 5.4km from city centre        2.0   
81              Hotel - 24.1km from city centre  No rating   
82              Hotel - 14.8km from city centre  No rating   
83   Bed and Breakfast - 2.9km from city centre        7.6   
84  Bed and Breakfast - 13.1km from city centre  No rating   

         total_reviews overall_reviews     privates_from_price  \
0    101 Total Reviews       Very Good   Privates From\nRs9100   
1    191 Total Reviews          Superb   No Privates Available   
2   1763 Total Reviews       Very Good   Privates From\nRs5786   
3    394 Total Reviews            Good  Privates From\nRs10110   
4     41 Total Reviews          Superb   No Privates Available   
..                 ...             ...                     ...   
80     2 Total Reviews          Rating  Privates From\nRs16257   
81   108 Total Reviews       No Rating   Privates From\nRs4550   
82    58 Total Reviews       No Rating   Privates From\nRs4348   
83   236 Total Reviews       Very Good   Privates From\nRs6471   
84    26 Total Reviews       No Rating   Privates From\nRs9100   

      dorms_from_price                                         facilities  
0   Dorms From\nRs1719  [Free WiFi, Follows Covid-19 sanitation guidance]  
1   Dorms From\nRs2932  [Free WiFi, Follows Covid-19 sanitation guidance]  
2   Dorms From\nRs2224  [Free WiFi, Free Breakfast, Follows Covid-19 s...  
3   Dorms From\nRs1517  [Free WiFi, Follows Covid-19 sanitation guidance]  
4   Dorms From\nRs3170                                        [Free WiFi]  
..                 ...                                                ...  
80  No Dorms Available                                        [Free WiFi]  
81  No Dorms Available  [Free Breakfast, Follows Covid-19 sanitation g...  
82  No Dorms Available  [Free Breakfast, Follows Covid-19 sanitation g...  
83  No Dorms Available  [Free WiFi, Follows Covid-19 sanitation guidance]  
84  No Dorms Available                                        [Free WiFi]  

[85 rows x 8 columns]